In [5]:
import os
import glob
import tifffile as tiff
from cellpose import denoise, models, io
from skimage import measure 
import pandas as pd

# For loop

In [6]:
def denoise_and_segment_image(img, img_name, output_path,  cellpose_model):
    """
    Perform denoising and segmentation on a 3D image using Cellpose.

    Parameters:
    - img (numpy.ndarray): Input image array to process.
    - img_name (str): Name of the input image (for logging and saving purposes).
    - output_path (str): Base path to save the output.
    - cellpose_model: Pre-initialized Cellpose segmentation model.

    Returns:
    None
    """
    # Display input image information
    print(f"Processing file: {img_name}, shape: {img.shape}")


    # Perform segmentation
    masks, flows, styles = cellpose_model.eval(
        x=img,  # Your input image array
        batch_size=8,
        diameter=31.0,
        channels=[2,1],
        do_3D=False,
        channel_axis = None,
    )
    
    print("Mask Segmentation Down!")

    # Create and save properties DataFrame
    props = measure.regionprops_table(masks, properties=['centroid'])
    props_df = pd.DataFrame(props)
    
    # Rename the columns
    props_df = props_df.rename(columns={
        'centroid-0': 'centroid_y',
        'centroid-1': 'centroid_x'
    })

    # Print number of rows (number of detected regions)
    print(f"Number of detected regions in {img_name}: {len(props_df)}")
    num_cells = len(props_df)

    # Create props CSV filename based on the original image name
    centroids_path = output_path + '_centroids.csv'

    props_df.to_csv(centroids_path, index=False)
    print(f"Properties saved to: {centroids_path}")

    # Save the masks as TIFF files
    io.save_masks(
        images=[img],
        masks=[masks],
        flows=[flows],
        file_names=[output_path],
        png=False,
        tif=True,
        channels=[0, 0]
    )

    io.masks_flows_to_seg(
        images=[img],  # List containing the image
        masks=[masks],            # List containing the mask
        flows=[flows],            # List containing the flows
        file_names=[output_path],  # Base filename for saving
        diams=31.0,              # Diameter used in Cellpose

    )

    print("Segmentation output saved to 'segmented_output_seg.npy'")

    return num_cells

In [7]:
# def process_images_in_folder(input_folder, output_folder,  cellpose_model):
#     """
#     Process all TIF images in a folder: denoise, segment, and save the output.

#     Parameters:
#     - input_folder (str): Path to the folder containing input images.
#     - output_folder (str): Path to save the processed output images.
#     - denoise_model: Pre-initialized denoising model.
#     - cellpose_model: Pre-initialized Cellpose segmentation model.

#     Returns:
#     None
#     """
#     i = 0

#     # Ensure the output folder exists
#     os.makedirs(output_folder, exist_ok=True)

#     # Find all TIF files in the input folder
#     file_paths = glob.glob(os.path.join(input_folder, "*.tif"))
    
#     total_cells = 0
#     for file_path in file_paths:
#         # Read the image
#         # print("file_path:", file_path)
#         img = tiff.imread(file_path)

#         # Get the base file name
#         file_name = os.path.basename(file_path)
#         # print("file_name:", file_name)

#         file_base = os.path.splitext(file_name)[0]
#         # print("file_base:", file_base)

#         file_base = file_base
#         # print("file_base2:", file_base)

#         # Define the output path for the current image
#         output_path = os.path.join(output_folder, file_base)
#         # print("output_path:", output_path)

#         # Print the file name
#         # print(f"Processing image {i}")
        
#         img_name = file_name
#         # Run the denoise and segmentation function
#         num_cells = denoise_and_segment_image(
#             img=img,
#             img_name=file_name,
#             output_path=output_path,   
#             cellpose_model=cellpose_model
#         )
#         total_cells += num_cells
#         i += 1

#         print(f"Output of [{img_name}] saved as: {output_path}\n")

#     return total_cells

In [8]:
def process_images_in_folder(input_folder, output_folder, cellpose_model):
    """
    Process all TIF images in a folder: denoise, segment, and save the output.

    Parameters:
    - input_folder (str): Path to the folder containing input images.
    - output_folder (str): Path to save the processed output images.
    - cellpose_model: Pre-initialized Cellpose segmentation model.

    Returns:
    int: Total number of cells detected across all processed images.
    """
    i = 0

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Find all TIF files in the input folder
    file_paths = glob.glob(os.path.join(input_folder, "*.tif"))
    
    total_cells = 0
    for file_path in file_paths:
        # Read the image
        img = tiff.imread(file_path)

        # Check if image has valid dimensions
        if 0 in img.shape:
            file_name = os.path.basename(file_path)
            print(f"Skipping {file_name} due to invalid dimensions: {img.shape}")
            continue

        # Get the base file name
        file_name = os.path.basename(file_path)
        file_base = os.path.splitext(file_name)[0]
        
        # Define the output path for the current image
        output_path = os.path.join(output_folder, file_base)
        
        img_name = file_name
        # Run the denoise and segmentation function
        num_cells = denoise_and_segment_image(
            img=img,
            img_name=file_name,
            output_path=output_path,   
            cellpose_model=cellpose_model
        )
        total_cells += num_cells
        i += 1

        print(f"Output of [{img_name}] saved as: {output_path}\n")

    return total_cells

In [9]:
ddir = 'G:\\Alexis_data\\Project\\SpatialOmics_cellpose\\results'
# base_path = "G:\Alexis_data\Project\SpatialOmics_cellpose\data\output-XETG00245__0034188__1782od__20240613__195055"
base_path = "G:\Alexis_data\Project\SpatialOmics_cellpose\data\output-XETG00245__0034188__1782os__20240613__195055"
sample_name = "1782os"

In [10]:
# model set up
# denoise_model = denoise.DenoiseModel(gpu=True, model_type='cyto3',  diam_mean=31.0)
model = models.CellposeModel(gpu=True, 
                             pretrained_model='G:/Alexis_data/Project/SpatialOmics_cellpose/models/HILT_Alexis_1113' ,
                             diam_mean=33.0
                             )

In [11]:

# Example usage
total = process_images_in_folder(
    input_folder = f'{ddir}/cropped_image/{sample_name}',      
    output_folder = f'{ddir}/segmenatation/{sample_name}',
    cellpose_model = model
)

print(f"Total cells detected: {total}")

Processing file: cropped_square_com_1.tif, shape: (2, 3011, 3011)
Mask Segmentation Down!
Number of detected regions in cropped_square_com_1.tif: 208
Properties saved to: G:\Alexis_data\Project\SpatialOmics_cellpose\results/segmenatation/1782os\cropped_square_com_1_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_1.tif] saved as: G:\Alexis_data\Project\SpatialOmics_cellpose\results/segmenatation/1782os\cropped_square_com_1

Processing file: cropped_square_com_10.tif, shape: (2, 3011, 2770)
Mask Segmentation Down!
Number of detected regions in cropped_square_com_10.tif: 325
Properties saved to: G:\Alexis_data\Project\SpatialOmics_cellpose\results/segmenatation/1782os\cropped_square_com_10_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_10.tif] saved as: G:\Alexis_data\Project\SpatialOmics_cellpose\results/segmenatation/1782os\cropped_square_com_10

Processing file: cropped_square_com_1

no masks found, will not save PNG or outlines


Mask Segmentation Down!
Number of detected regions in cropped_square_com_2.tif: 0
Properties saved to: G:\Alexis_data\Project\SpatialOmics_cellpose\results/segmenatation/1782os\cropped_square_com_2_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_2.tif] saved as: G:\Alexis_data\Project\SpatialOmics_cellpose\results/segmenatation/1782os\cropped_square_com_2

Processing file: cropped_square_com_3.tif, shape: (2, 3011, 3011)
Mask Segmentation Down!
Number of detected regions in cropped_square_com_3.tif: 346
Properties saved to: G:\Alexis_data\Project\SpatialOmics_cellpose\results/segmenatation/1782os\cropped_square_com_3_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_3.tif] saved as: G:\Alexis_data\Project\SpatialOmics_cellpose\results/segmenatation/1782os\cropped_square_com_3

Processing file: cropped_square_com_4.tif, shape: (2, 3011, 3011)
Mask Segmentation Down!
Number of detected r